### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy.analyses import Pf1D
from neuropy import plotting
import subjects

### Distribution of rank correlations in POST

In [ ]:
import pingouin as pg

sessions = subjects.pf_sess()

for sub, sess in enumerate(sessions):
    print(sess.animal.name)
    neurons = sess.neurons_stable.get_neuron_type(neuron_type="pyr")
    position = sess.maze

    pbe_epochs = sess.pbe
    pbe_bins = pbe_epochs.flatten()
    # n_spikes = [np.histogram(_,bins=pbe_bins)[0][::2] for _ in neurons.spiketrains]
    # good_bool = (np.array(n_spikes)>0).sum(axis=0)>=5

    # pbe_epochs = pbe_epochs[good_bool]

    print(f"#Epochs: {len(pbe_epochs)}")
    metadata = {"score_method": "wcorr"}

    score_df = []
    for direction in ["up",'down']:
        pf = Pf1D(
            neurons=neurons,
            position=position,
            sigma=4,
            grid_bin=2,
            epochs=sess.maze_run[direction],
            frate_thresh=0.3,
        )
        # sorted_neuron_id = pf.get_sort_order(by='neuron_id')
        pf_neurons = neurons.get_by_id(pf.neuron_ids)
        tc = pf.tuning_curves
        peak_locs = np.argmax(tc,axis=1)

        mean_spike = np.array(
            [
                stats.binned_statistic(_, _, bins=pbe_bins, statistic="mean")[0][::2]
                for _ in pf_neurons.spiketrains
            ]
        )
        latency_df = pd.DataFrame(np.hstack((peak_locs[:,np.newaxis],mean_spike)))
        latency_df.columns = latency_df.columns.astype('str')
        corr = pg.pairwise_corr(data=latency_df,columns='0',method='spearman')


        df = pd.DataFrame(
            dict(
                rank_corr=corr.r.values,
                pval=corr['p-unc'].values,
            )
        )

        score_df.append(df.add_prefix(direction + "_"))

    score_df = pd.concat(score_df, axis=1)
    new_epochs = pbe_epochs.add_dataframe(score_df)
    new_epochs.metadata = metadata
    new_epochs.save(sess.filePrefix.with_suffix(".pbe.replay.spearman"))




In [ ]:
mean_spike

### Spearman rank correlation distribution

In [ ]:
# sessions = subjects.pf_sess()
sessions = subjects.nsd.ratNday2

radon_df = []
for sub, sess in enumerate(sessions):

    neurons = sess.neurons_stable.get_neuron_type('pyr')
    pbe_epochs = sess.pbe.flatten()
    pbe = sess.pbe.to_dataframe()
    pbe_peak = pbe.peak_time
    replay_pbe = sess.replay_spearman.to_dataframe()

    n_spikes = [np.histogram(_,bins=pbe_epochs)[0][::2] for _ in neurons.spiketrains]
    fire_bool = (np.array(n_spikes)>0).sum(axis=0)>=5

    # pbe_spikes = neurons.get_spikes_in_epochs(sess.pbe,bin_size=0.02)[0]
    # pbe_spikes = np.array([np.all(arr.sum(axis=0)>0) for arr in pbe_spikes])


    # fire_bool = fire_bool & pbe_spikes
    # rpl_epochs = sess.ripple.flatten()
    # pbe_bool = np.digitize(pbe_peak, rpl_epochs)%2==1
    replay_pbe= replay_pbe[fire_bool]

    starts = replay_pbe.start.values
    up_score = replay_pbe.up_rank_corr.values
    up_pval = replay_pbe.up_pval.values
    down_score = replay_pbe.down_rank_corr.values
    down_pval = replay_pbe.down_pval.values

    best_bool = np.abs(up_score) > np.abs(down_score)
    score = np.zeros(len(starts))
    score[best_bool] = up_score[best_bool]
    score[~best_bool] = down_score[~best_bool]

    pval = np.zeros(len(starts))
    pval[best_bool] = up_pval[best_bool]
    pval[~best_bool] = down_pval[~best_bool]


    epochs = sess.get_zt_epochs(include_maze=True)

    period= epochs['PRE'].flatten()
    period_indx = (starts > period[0]) & (starts < period[1])
    mean_period = score[period_indx].mean()
    
    for i, e in enumerate(epochs.itertuples()):
        indx = (starts > e.start) & (starts < e.stop)
        score_e = score[indx]
        pval_e = pval[indx]
        radon_df.append(
            pd.DataFrame(
                {
                    'zt':e.label,
                    'score':np.abs(score_e)[pval_e<0.05],
                    'session':sub,
                    "grp": sess.tag,
                }
            )
        )

radon_df = pd.concat(radon_df, ignore_index=True)


In [ ]:
_,ax = plt.subplots()

sns.violinplot(data=radon_df,x='zt',y='score',hue='grp')